# IMPORTING LIBRARIES

In [1]:
import googlemaps
import pandas as pd
import pickle
from geopy.geocoders import Nominatim

# FUNTION FOR GETTING VILLAGE COORDINATES

In [3]:
def get_village_coordinates(village_name, state_name):
    
    try:
        # Initialize the geolocator with a custom user agent
        geolocator = Nominatim(user_agent="village_locator")
        # Use the geolocator to find the location based on the village and state name
        location = geolocator.geocode(f"{village_name}, {state_name}, India") 
        if location:
            return location.latitude, location.longitude
        else:
            return None 
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# FUNCTION FOR GETTING VILLAGE INFRASTRUCTURE DETAILS

In [5]:
def get_village_infrastructure_details(coordinates, api_key):
    
    # Initialize Google Maps client
    gmaps = googlemaps.Client(key=api_key)
    
    # Extract latitude and longitude
    latitude = coordinates[0]
    longitude = coordinates[1]
    
    # Define needed infrastructure and their respective keywords
    needed_infrastructure = ['library', 'primary_school', 'secondary_school', 'college', 'atm', 'supermarket', 
                              'panchayat_office', 'post_office', 'hospital', 'bus_stop', 'common_service_center']
    
    infrastructure_keywords = {
        'primary_school': 'elementary school|mpp|government primary|state board school|cbse school|mandal parishad|govt primary',
        'secondary_school': 'middle school|zph|government high|high school|state board school|zp high|zilla parishad|cbse school|govt high',
        'college': 'junior college|government junior|jr college|govt jr|govt junior',
        'supermarket': 'supermarket|super market|grocery store|hyper store',
        'panchayat_office': 'panchayat|sachivalayam',
        'post_office': 'post',
        'common_service_center': 'csc|common'
    }
    
    # Initialize the results dictionary
    infras_results = {}
    
    # Check for each infrastructure
    for infras in needed_infrastructure:
        try:
            if infras in infrastructure_keywords:
                places = gmaps.places_nearby(
                    location=(latitude, longitude), 
                    radius=5000, 
                    keyword=infrastructure_keywords[infras]
                )
            else:
                places = gmaps.places_nearby(
                    location=(latitude, longitude), 
                    radius=5000, 
                    type=infras
                )
            
            # Update results with 1 if results are found, otherwise 0
            infras_results[infras] = 1 if places['results'] else 0
        
        except Exception as e:
            # Log the exception and mark as not found
            print(f"Error checking {infras}: {e}")
            infras_results[infras] = 0
    
    return infras_results

# PREDICTING NEEDED INFRASTRUCTURE FOR THE VILLAGE

In [7]:
village= input("Enter your village name: ")
state= input("Enter your state name: ")
village_coordinates= get_village_coordinates(village, state)
api_key= "google-maps-api-key"
infrastructure_found_in_village= get_village_infrastructure_details(village_coordinates, api_key)

# list of infrastructure details with binary representation
features= list(infrastructure_found_in_village.values())

# Load the trained model
with open('rftrained_model.pkl', 'rb') as f:
  trained_model = pickle.load(f)

all_infrastructure = ['library', 'primary_school', 'secondary_school', 'college', 'atm', 'supermarket', 
                    'panchayat_office', 'post_office', 'hospital', 'bus_stop', 'common_service_center'] 

# Predict the development level
predicted_development = trained_model.predict([features])[0]

if predicted_development == 1:  # Fully developed
    print("No need of any infrastructure")
else:
    needed_infrastructure = []
    for i, infra in enumerate(all_infrastructure):
        if features[i] == 0:
            needed_infrastructure.append(infra)
    print(f"Needed infrastructure: {', '.join(needed_infrastructure)}")

Needed infrastructure: Hospital, Post_office, Library, Bus_stop


C:\Users\Saikiran Rai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
